# MCP Features: Tools, Resources, and Prompts

This example uses the FastMCP client to interact with an MCP server that exposes tools, resources, and prompts. We explore each feature type through the client's programmatic API.

## Prerequisites

Before running this notebook, start the MCP server in a separate terminal:

```bash
cd agentic_patterns/mcp
fastmcp run example_mcp_server_v2.py
```

The server will start on `http://127.0.0.1:8000`. Keep it running while executing the cells below.

In [1]:
from fastmcp import Client

## Connect to the MCP Server

The client connects to the running server via HTTP transport.

In [2]:
client = Client("http://127.0.0.1:8000/mcp")

## Tools

Tools are callable functions exposed by the server. The client can discover them and invoke them with arguments.

In [3]:
async with client:
    # Discover available tools
    tools = await client.list_tools()
    for tool in tools:
        print(f"Tool: {tool.name}")
        print(f"  Description: {tool.description}")
        print(f"  Schema: {tool.inputSchema}")

Tool: add
  Description:  Add two numbers 
  Schema: {'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'}


In [4]:
async with client:
    # Call the 'add' tool
    result = await client.call_tool("add", {"a": 40, "b": 2})
    print(f"Result: {result}")

Result: CallToolResult(content=[TextContent(type='text', text='42', annotations=None, meta=None)], structured_content={'result': 42}, meta=None, data=addOutput(result=42), is_error=False)


## Resources

Resources expose data through URI-addressable endpoints. They can be static or templated with parameters.

In [5]:
async with client:
    # Discover resource templates
    templates = await client.list_resource_templates()
    for template in templates:
        print(f"Template: {template.uriTemplate}")
        print(f"  Description: {template.description}")

Template: postgres://{database}/{table_name}/schema
  Description: Get a table's schema


In [6]:
async with client:
    # Read a resource by filling in the template parameters
    content = await client.read_resource("postgres://mydb/users/schema")
    print(f"Resource content:\n{content[0].text}")

Resource content:

        -- This is the schema for the users table
        -- in the mydb database
        CREATE TABLE users (id INT, name VARCHAR(255));
        


## Prompts

Prompts are server-defined instruction templates. The client requests them by name with arguments, and the server returns the rendered prompt text.

In [7]:
async with client:
    # Discover available prompts
    prompts = await client.list_prompts()
    for prompt in prompts:
        print(f"Prompt: {prompt.name}")
        print(f"  Description: {prompt.description}")
        print(f"  Arguments: {prompt.arguments}")

Prompt: hello_prompt
  Description: Get a personalized greeting
  Arguments: [PromptArgument(name='name', description=None, required=True)]


In [8]:
async with client:
    # Get a rendered prompt
    result = await client.get_prompt("hello_prompt", {"name": "Alice"})
    for msg in result.messages:
        print(f"Role: {msg.role}")
        print(f"Content: {msg.content.text}")

Role: user
Content: Your name is Alice and you are a helpful assistant.
